In [1]:
%matplotlib widget

In [2]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json

import os

In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [4]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

# input json name

In [5]:
file_parameters['watershed'] = 'yeongsan'

In [6]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [7]:
pd.set_option('display.max_columns', 1000)

In [8]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [9]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [10]:
gain_epochs , rnn_epochs

(2000, 10)

# run file

In [11]:
run_num = range(len(folder))


run_num = [0]




real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)


# Gain Training Flag

In [12]:
__GAIN_TRAINING__ = False

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])

    df_all, train_mean, train_std, df = normalize(df)


    

    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/yeong/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['나주_2015.xlsx', '나주_2016.xlsx', '나주_2017.xlsx', '나주_2018.xlsx', '나주_2019.xlsx', '나주_2020.xlsx'], ['지석천_2015.xlsx', '지석천_2016.xlsx', '지석천_2017.xlsx', '지석천_2018.xlsx', '지석천_2019.xlsx', '지석천_2020.xlsx'], ['용봉_2015.xlsx', '용봉_2016.xlsx', '용봉_2017.xlsx', '용봉_2018.xlsx', '용봉_2019.xlsx', '용봉_2020.xlsx'], ['우치_2015.xlsx', '우치_2016.xlsx', '우치_2017.xlsx', '우치_2018.xlsx', '우치_2019.xlsx', '우치_2020.xlsx']]
data/yeong/자동/나주_2015.xlsx
data/yeong/자동/나주_2016.xlsx
data/yeong/자동/나주_2017.xlsx
data/yeong/자동/나주_2018.xlsx
data/yeong/자동/나주_2019.xlsx
data/yeong/자동/나주_2020.xlsx
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/yeong/자동/지석천_2015.xlsx
data/yeong/자동/지석천_2016.xlsx
data/yeong/자동/지석천_2017.xlsx
data/yeong/자동/지석천_2018.xlsx
data/yeong/자동/지석천_2019.xlsx
data/yeong/자동/지석천_2020.xlsx
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/yeong/자동/용봉_2015.xlsx
data/

In [14]:
print(real_df_all.shape)

(52608, 48)


In [15]:
def dataset_slice(df, train_ratio, val_ratio, test_ratio):
    total_no = df.shape[0]

    train_no = int(total_no * train_ratio)
    #
    val_no = int(total_no * (train_ratio + val_ratio))

    # val_no = int(total_no*val_ratio)
    # train_no =int(total_no * (train_ratio+val_ratio))
    #
    # val_slice = slice(0, val_no)
    # train_slice = slice(val_no, train_no)
    # test_slice = slice(train_no, None)

    train_slice = slice(0, train_no)    #0.8
    val_slice = slice(train_no, val_no)   #0.1
    test_slice = slice(val_no, None)
    # val_slice = slice(train_no, val_no)   #0.1
    # test_slice = slice(val_no, None)
#     train_slice = slice(0, train_no)  # 0.8
#     val_slice = slice(train_no, None)  # 0.1
#     test_slice = slice(val_no, None)

    train = pd.DataFrame(df[train_slice])
    val = pd.DataFrame(df[val_slice])
    test = pd.DataFrame(df[test_slice])

    test_slice2 = slice(total_no - 288, None)
    test2 = pd.DataFrame(df[test_slice2])

    #print('total_no :1111111111111111111111111 ')
    #print('total_no : ', total_no)
    #print('train : ', train.shape)
    #print('val : ', val.shape)
    #print('test : ', test.shape)


    return train, val, test, test2

# slice DF

In [46]:
train_df, val_df, test_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)

In [47]:
val_df.shape, test_df.shape, train_df.shape

((5261, 48), (5261, 48), (42086, 48))

In [48]:
print('-------------------prediction')
print('-------------------prediction')
print('-------------------prediction')

print('real_df_all.type : ', type(real_df_all))
print('train_df.type : ', type(train_df))
print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)


-------------------prediction
-------------------prediction
-------------------prediction
real_df_all.type :  <class 'pandas.core.frame.DataFrame'>
train_df.type :  <class 'pandas.core.frame.DataFrame'>
train_df.shape :  (42086, 48) val_df.shape :  (5261, 48) test_df.shape: (5261, 48)


# target column

In [49]:

label_columns_indices = {name: i for i, name in enumerate(dfff[0])}

print("label_columns_indices:")
print(label_columns_indices)


target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

rnn_target_column = "toc"

print('target columns : ', rnn_target_column)
num_features = dfff[0].shape[1]



target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)


label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target columns :  toc
target_col_idx :  4
out_num_features :  1


In [50]:
val_nse = {}
val_pbias = {}


WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

if __RNN_TRAINING__:
    if not os.path.exists('save/' + watershed):
        os.makedirs('save/' + watershed)


idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]

indices = {name: i for i, name in enumerate(idx)}

model_path = "save/" + watershed + "models/" + pa[indices[target_col_idx]]
print("save model path : ", model_path)

val_nse = {}
val_pbias = {}

 # +"gru.ckpt" -- path




save model path :  save/yeong/models/toc/


In [51]:
plt.figure()
plt.plot(real_df_all.iloc[:,out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:
plt.figure()
plt.plot(test_df.iloc[:,out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 모델 학습

In [38]:
__RNN_TRAINING__ = True

In [53]:
rnn_epochs = 500
# multi_linear_model = model_multi_linear(
#     window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
#     #training_flag=__RNN_TRAINING__, checkpoint_path="save/"+watershed+"models/multi_linear.ckpt")
#     training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_linear.ckpt")
# elman_model = model_elman(
#     window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
#     training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"elman.ckpt")
gru_model = model_gru(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")
# multi_lstm_model = model_multi_lstm(
#     window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
#     training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_lstm.ckpt")
# multi_conv_model = model_multi_conv(
#     window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
#     training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_conv.ckpt")


Epoch 1/500
1/1 [==============================] - 3s 3s/step - loss: 0.7831 - mean_absolute_error: 0.6862 - nse: -0.1797 - val_loss: 0.9133 - val_mean_absolute_error: 0.6109 - val_nse: -0.3994

Epoch 00001: val_loss improved from inf to 0.91330, saving model to save/yeong/models/toc/gru.ckpt
Epoch 2/500
1/1 [==============================] - 1s 593ms/step - loss: 0.7732 - mean_absolute_error: 0.6516 - nse: -0.1284 - val_loss: 0.7903 - val_mean_absolute_error: 0.5655 - val_nse: -0.2626

Epoch 00002: val_loss improved from 0.91330 to 0.79026, saving model to save/yeong/models/toc/gru.ckpt
Epoch 3/500
1/1 [==============================] - 1s 569ms/step - loss: 0.8140 - mean_absolute_error: 0.6843 - nse: -0.1403 - val_loss: 0.6731 - val_mean_absolute_error: 0.5163 - val_nse: -0.1697

Epoch 00003: val_loss improved from 0.79026 to 0.67313, saving model to save/yeong/models/toc/gru.ckpt
Epoch 4/500
1/1 [==============================] - 1s 518ms/step - loss: 1.0778 - mean_absolute_error: 0


Epoch 00030: val_loss did not improve from 0.34877
Epoch 31/500
1/1 [==============================] - 1s 506ms/step - loss: 0.2982 - mean_absolute_error: 0.3925 - nse: 0.5843 - val_loss: 0.3434 - val_mean_absolute_error: 0.4085 - val_nse: 0.2575

Epoch 00031: val_loss improved from 0.34877 to 0.34341, saving model to save/yeong/models/toc/gru.ckpt
Epoch 32/500
1/1 [==============================] - 1s 521ms/step - loss: 0.2958 - mean_absolute_error: 0.3706 - nse: 0.6359 - val_loss: 0.4349 - val_mean_absolute_error: 0.4319 - val_nse: 0.3103

Epoch 00032: val_loss did not improve from 0.34341
Epoch 33/500
1/1 [==============================] - 1s 538ms/step - loss: 0.3013 - mean_absolute_error: 0.3734 - nse: 0.6063 - val_loss: 0.5231 - val_mean_absolute_error: 0.4921 - val_nse: 0.3870

Epoch 00033: val_loss did not improve from 0.34341
Epoch 34/500
1/1 [==============================] - 1s 540ms/step - loss: 0.2399 - mean_absolute_error: 0.3477 - nse: 0.6594 - val_loss: 0.3664 - val_me

1/1 [==============================] - 1s 508ms/step - loss: 0.2525 - mean_absolute_error: 0.3433 - nse: 0.6613 - val_loss: 0.3846 - val_mean_absolute_error: 0.3903 - val_nse: 0.4394

Epoch 00063: val_loss did not improve from 0.29954
Epoch 64/500
1/1 [==============================] - 1s 558ms/step - loss: 0.2683 - mean_absolute_error: 0.3498 - nse: 0.6765 - val_loss: 0.3032 - val_mean_absolute_error: 0.3665 - val_nse: 0.4562

Epoch 00064: val_loss did not improve from 0.29954
Epoch 65/500
1/1 [==============================] - 1s 591ms/step - loss: 0.2127 - mean_absolute_error: 0.3180 - nse: 0.6980 - val_loss: 0.3223 - val_mean_absolute_error: 0.3861 - val_nse: 0.4186

Epoch 00065: val_loss did not improve from 0.29954
Epoch 66/500
1/1 [==============================] - 0s 499ms/step - loss: 0.3255 - mean_absolute_error: 0.3796 - nse: 0.5872 - val_loss: 0.4211 - val_mean_absolute_error: 0.4142 - val_nse: 0.4417

Epoch 00066: val_loss did not improve from 0.29954
Epoch 67/500
1/1 [===


Epoch 00095: val_loss did not improve from 0.25845
Epoch 96/500
1/1 [==============================] - 1s 540ms/step - loss: 0.2438 - mean_absolute_error: 0.3322 - nse: 0.6843 - val_loss: 0.3507 - val_mean_absolute_error: 0.3921 - val_nse: 0.4514

Epoch 00096: val_loss did not improve from 0.25845
Epoch 97/500
1/1 [==============================] - 0s 493ms/step - loss: 0.2482 - mean_absolute_error: 0.3426 - nse: 0.6204 - val_loss: 0.3776 - val_mean_absolute_error: 0.4253 - val_nse: 0.4130

Epoch 00097: val_loss did not improve from 0.25845
Epoch 98/500
1/1 [==============================] - 1s 527ms/step - loss: 0.2700 - mean_absolute_error: 0.3349 - nse: 0.6749 - val_loss: 0.4033 - val_mean_absolute_error: 0.4552 - val_nse: 0.3431

Epoch 00098: val_loss did not improve from 0.25845
Epoch 99/500
1/1 [==============================] - 1s 539ms/step - loss: 0.2243 - mean_absolute_error: 0.3355 - nse: 0.6498 - val_loss: 0.3837 - val_mean_absolute_error: 0.4422 - val_nse: 0.4117

Epoch 0


Epoch 00128: val_loss did not improve from 0.25845
Epoch 129/500
1/1 [==============================] - 1s 544ms/step - loss: 0.2580 - mean_absolute_error: 0.3547 - nse: 0.6439 - val_loss: 0.3640 - val_mean_absolute_error: 0.4203 - val_nse: 0.3661

Epoch 00129: val_loss did not improve from 0.25845
Epoch 130/500
1/1 [==============================] - 1s 545ms/step - loss: 0.3497 - mean_absolute_error: 0.4062 - nse: 0.6044 - val_loss: 0.3334 - val_mean_absolute_error: 0.3739 - val_nse: 0.4728

Epoch 00130: val_loss did not improve from 0.25845
Epoch 131/500
1/1 [==============================] - 1s 535ms/step - loss: 0.2470 - mean_absolute_error: 0.3510 - nse: 0.6761 - val_loss: 0.3319 - val_mean_absolute_error: 0.3778 - val_nse: 0.3980

Epoch 00131: val_loss did not improve from 0.25845
Epoch 132/500
1/1 [==============================] - 1s 547ms/step - loss: 0.2474 - mean_absolute_error: 0.3420 - nse: 0.6726 - val_loss: 0.3638 - val_mean_absolute_error: 0.3751 - val_nse: 0.4314

Epo

...


Epoch 00359: val_loss did not improve from 0.25013
Epoch 360/500
1/1 [==============================] - 1s 501ms/step - loss: 0.2071 - mean_absolute_error: 0.3156 - nse: 0.7343 - val_loss: 0.5728 - val_mean_absolute_error: 0.5085 - val_nse: 0.2968

Epoch 00360: val_loss did not improve from 0.25013
Epoch 361/500
1/1 [==============================] - 1s 520ms/step - loss: 0.1621 - mean_absolute_error: 0.2964 - nse: 0.8004 - val_loss: 0.4681 - val_mean_absolute_error: 0.4756 - val_nse: 0.2629

Epoch 00361: val_loss did not improve from 0.25013
Epoch 362/500
1/1 [==============================] - 1s 537ms/step - loss: 0.1898 - mean_absolute_error: 0.2964 - nse: 0.7570 - val_loss: 0.4699 - val_mean_absolute_error: 0.4860 - val_nse: 0.1418

Epoch 00362: val_loss did not improve from 0.25013
Epoch 363/500
1/1 [==============================] - 1s 536ms/step - loss: 0.1695 - mean_absolute_error: 0.2916 - nse: 0.7227 - val_loss: 0.4809 - val_mean_absolute_error: 0.4814 - val_nse: 0.1944

Epo


Epoch 00392: val_loss did not improve from 0.25013
Epoch 393/500
1/1 [==============================] - 1s 517ms/step - loss: 0.1662 - mean_absolute_error: 0.2816 - nse: 0.7468 - val_loss: 0.4420 - val_mean_absolute_error: 0.4591 - val_nse: 0.2752

Epoch 00393: val_loss did not improve from 0.25013
Epoch 394/500
1/1 [==============================] - 1s 519ms/step - loss: 0.2091 - mean_absolute_error: 0.3060 - nse: 0.6990 - val_loss: 0.4801 - val_mean_absolute_error: 0.4854 - val_nse: 0.2120

Epoch 00394: val_loss did not improve from 0.25013
Epoch 395/500
1/1 [==============================] - 1s 548ms/step - loss: 0.1709 - mean_absolute_error: 0.2939 - nse: 0.7895 - val_loss: 0.5422 - val_mean_absolute_error: 0.5092 - val_nse: 0.1997

Epoch 00395: val_loss did not improve from 0.25013
Epoch 396/500
1/1 [==============================] - 1s 541ms/step - loss: 0.1739 - mean_absolute_error: 0.2938 - nse: 0.7707 - val_loss: 0.3988 - val_mean_absolute_error: 0.4467 - val_nse: 0.3386

Epo


Epoch 00425: val_loss did not improve from 0.25013
Epoch 426/500
1/1 [==============================] - 1s 529ms/step - loss: 0.1723 - mean_absolute_error: 0.2880 - nse: 0.7783 - val_loss: 0.5577 - val_mean_absolute_error: 0.4966 - val_nse: 0.2083

Epoch 00426: val_loss did not improve from 0.25013
Epoch 427/500
1/1 [==============================] - 1s 608ms/step - loss: 0.1689 - mean_absolute_error: 0.2945 - nse: 0.7519 - val_loss: 0.4542 - val_mean_absolute_error: 0.4736 - val_nse: 0.2637

Epoch 00427: val_loss did not improve from 0.25013
Epoch 428/500
1/1 [==============================] - 1s 567ms/step - loss: 0.1555 - mean_absolute_error: 0.2921 - nse: 0.7751 - val_loss: 0.4752 - val_mean_absolute_error: 0.4802 - val_nse: 0.2418

Epoch 00428: val_loss did not improve from 0.25013
Epoch 429/500
1/1 [==============================] - 1s 522ms/step - loss: 0.2149 - mean_absolute_error: 0.3232 - nse: 0.7462 - val_loss: 0.5655 - val_mean_absolute_error: 0.5427 - val_nse: 0.1883

Epo


Epoch 00458: val_loss did not improve from 0.25013
Epoch 459/500
1/1 [==============================] - 1s 506ms/step - loss: 0.1626 - mean_absolute_error: 0.2742 - nse: 0.7682 - val_loss: 0.7261 - val_mean_absolute_error: 0.5807 - val_nse: -0.1582

Epoch 00459: val_loss did not improve from 0.25013
Epoch 460/500
1/1 [==============================] - 1s 545ms/step - loss: 0.1624 - mean_absolute_error: 0.2871 - nse: 0.7594 - val_loss: 0.6504 - val_mean_absolute_error: 0.5412 - val_nse: -0.1049

Epoch 00460: val_loss did not improve from 0.25013
Epoch 461/500
1/1 [==============================] - 1s 549ms/step - loss: 0.1433 - mean_absolute_error: 0.2617 - nse: 0.7602 - val_loss: 0.8220 - val_mean_absolute_error: 0.6029 - val_nse: 0.0387

Epoch 00461: val_loss did not improve from 0.25013
Epoch 462/500
1/1 [==============================] - 1s 540ms/step - loss: 0.1842 - mean_absolute_error: 0.3002 - nse: 0.7167 - val_loss: 0.5073 - val_mean_absolute_error: 0.4733 - val_nse: 0.1092

E


Epoch 00491: val_loss did not improve from 0.25013
Epoch 492/500
1/1 [==============================] - 1s 507ms/step - loss: 0.1823 - mean_absolute_error: 0.2823 - nse: 0.7710 - val_loss: 0.5029 - val_mean_absolute_error: 0.5013 - val_nse: -0.0282

Epoch 00492: val_loss did not improve from 0.25013
Epoch 493/500
1/1 [==============================] - 1s 563ms/step - loss: 0.1549 - mean_absolute_error: 0.2706 - nse: 0.7929 - val_loss: 0.7787 - val_mean_absolute_error: 0.5879 - val_nse: 0.0470

Epoch 00493: val_loss did not improve from 0.25013
Epoch 494/500
1/1 [==============================] - 1s 544ms/step - loss: 0.1760 - mean_absolute_error: 0.2836 - nse: 0.7606 - val_loss: 0.4932 - val_mean_absolute_error: 0.4760 - val_nse: 0.0964

Epoch 00494: val_loss did not improve from 0.25013
Epoch 495/500
1/1 [==============================] - 1s 523ms/step - loss: 0.1692 - mean_absolute_error: 0.2877 - nse: 0.7895 - val_loss: 0.4111 - val_mean_absolute_error: 0.4245 - val_nse: 0.2763

Ep

## core / window.py / 

In [54]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    print(df.shape)
    print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
   
    for i in range(0,length,24):
#         print('i = ', i)
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
#     print('labels.mean(axis=1)')
#     print(labels.mean(axis=1).shape)
    print(inputs.shape)
    print(labels.shape)

    predictions = model(inputs)
    print(predictions.shape)
    
    predictions = predictions.numpy() * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    print('predictions.shape =', predictions.shape)
#     pred_day = predictions
    pred_day = hour_to_day_mean(predictions)
    print('pred_day.shape =', pred_day.shape)
#     label_day= labels
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
#     inputs_day = inputs_target
    inputs_day = hour_to_day_mean(inputs_target)
    
    #plt.figure(figsize=(10, 800))
    
#     input_index = np.array(range(0,length,24))
#     label_index = input_index + 24*7
    
#     print(input_index)
#     print(label_index)
    
    plt.figure()
    plt.plot(label_day[:, predict_day, :], label='label')
    plt.plot(pred_day[:, predict_day, :], label='pred')
#     plt.plot(input_index, inputs_day[:, 0, :], label='input')
#     plt.plot(label_index, label_day[:, 0, :], label='label')
#     plt.plot(label_index, pred_day[:, 0, :], label='pred')
    plt.legend()
        
        
        
#     o1 = np.mean(labels)
# #     print('labels =', labels)
#     print('o1 =', o1)
#     nse1 = ((labels - predictions)**2).sum()
#     nse2 = ((labels - o1)**2).sum()
#     nse3 = 1 - (nse1/nse2)
    
    
    
#     print('pred_day.shape =',pred_day.shape)
#     print('label_day.shape =',label_day.shape)
    #print('label_index =', label_index)
    
    o1 = np.mean(label_day[:,predict_day,0])
#     o1 = np.mean(label_day)
#     print('labels =', labels)
#     o1 = target_mean[plot_col]
    print('o1 =', o1)
    nse1 = ((pred_day-label_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('nse3 =', nse3)
    #mean = np.mean(label_day[predict_day])
    print('label_day[predict_day].shape =', label_day[predict_day].shape)
#     print(type(pred_day))
#     print(pred_day[:20, 0, 0])
#     print(label_day[:20, 0, 0])
    o = label_day[:, 0, 0]
    s = pred_day[:, 0, 0]
    mean = np.mean(o)
    print('mean =', mean)
    nse = 1. - np.sum((o-s)**2)/np.sum((o-mean)**2)
#     print('nse =', nse)
    
    
#     pbias1 = (label_day - pred_day).sum(axis=0)
#     pbias2 = (label_day).sum(axis=0)
#     pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
#     o1 = np.mean(label_day)
# #     print('labels =', labels)
#     print('o1 =', o1)
#     nse1 = ((label_day - pred_day)**2).sum()
#     nse2 = ((label_day - o1)**2).sum()
#     nse3 = 1 - (nse1/nse2)
#     print('nse3 =', nse3)
    #mean = np.mean(label_day[predict_day])
#     print(type(pred_day))
#     print(pred_day[:20, 0, 0])
#     print(label_day[:20, 0, 0])
#     o = label_day[:, 0, 0]
#     s = pred_day[:, 0, 0]
#     mean = np.mean(o)
#     print('mean =', mean)
#     nse = 1. - np.sum((o-s)**2)/np.sum((o-mean)**2)
#     print('nse =', nse)
    
    
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    
    
    return nse3, np.abs(pbias3), pred_day, labels
    

In [55]:
test_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
47347,0.612003,-1.087630,-1.545484,-0.221104,0.505913,0.142707,-0.627064,-0.596957,-1.366022,0.366025,0.820100,-1.152420,0.822589,0.967944,0.081753,-0.919043,-0.427025,-0.722573,0.236967,-0.545296,-1.366022,0.366025,0.820100,-1.152420,0.698242,-1.087630,-1.891728,0.001012,1.232818,0.274933,1.610440,-0.308300,-1.366022,0.366025,0.820100,-1.152420,0.760415,-0.144088,-0.003161,-0.732465,0.435546,-0.877776,0.577746,-0.680867,-1.366022,0.366025,0.820100,-1.152420
47348,0.596961,-1.087630,-1.597477,-0.204322,0.407976,0.145094,-0.621685,-0.586996,-1.224742,0.707105,0.819274,-1.153007,0.739357,0.653434,-0.120316,-0.913120,-0.398001,-0.772184,0.199507,-0.554331,-1.224742,0.707105,0.819274,-1.153007,0.693229,-1.087630,-1.841324,0.014832,1.307537,0.325482,1.626575,-0.314555,-1.224742,0.707105,0.819274,-1.153007,0.739357,-0.144088,-0.118047,-0.697914,0.552339,-0.826665,0.532996,-0.680358,-1.224742,0.707105,0.819274,-1.153007
47349,0.583924,-1.087630,-1.636528,-0.206297,0.506635,0.125953,-0.659335,-0.593251,-0.999998,0.999998,0.818447,-1.153594,0.655123,0.271515,-0.338054,-0.904235,-0.366810,-0.797088,0.092685,-0.557111,-0.999998,0.999998,0.818447,-1.153594,0.678187,-1.087630,-1.841779,0.010884,1.306815,0.324544,1.303859,-0.322432,-0.999998,0.999998,0.818447,-1.153594,0.710276,-0.200255,-0.230889,-0.692978,0.607479,-0.673524,0.529338,-0.681261,-0.999998,0.999998,0.818447,-1.153594
47350,0.567880,-1.087630,-1.687161,-0.213207,0.437723,0.112939,-0.675471,-0.599274,-0.707105,1.224742,0.817620,-1.154180,0.576905,-0.031766,-0.535584,-0.890414,-0.461116,-0.840336,0.111462,-0.558732,-0.707105,1.224742,0.817620,-1.154180,0.670164,-1.087630,-1.817258,0.071102,1.227743,0.423772,1.271587,-0.363205,-0.707105,1.224742,0.817620,-1.154180,0.666154,-0.278879,-0.416385,-0.741350,0.524777,-0.738114,0.509545,-0.685385,-0.707105,1.224742,0.817620,-1.154180
47351,0.544815,-1.087630,-1.741652,-0.242823,0.397817,0.120712,-0.675471,-0.611321,-0.366025,1.366022,0.816793,-1.154766,0.506710,-0.222712,-0.698602,-0.873632,-0.479258,-0.848851,-0.045601,-0.559196,-0.366025,1.366022,0.816793,-1.154766,0.664148,-1.087630,-1.813852,0.122435,1.066692,0.446800,1.196286,-0.419732,-0.366025,1.366022,0.816793,-1.154766,0.618020,-0.379973,-0.609828,-0.767017,0.703239,-0.715272,0.471895,-0.687493,-0.366025,1.366022,0.816793,-1.154766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52603,-0.929680,0.488478,1.292970,0.292406,0.026550,0.219398,-0.253954,0.113919,-1.366022,0.366025,0.010318,1.413645,-0.930236,0.486999,1.294677,0.292136,0.026659,0.221538,-0.255042,0.114701,-1.366022,0.366025,0.010318,1.413645,-0.930485,0.487197,1.294038,0.290977,0.027859,0.221296,-0.253660,0.115725,-1.366022,0.366025,0.010318,1.413645,-0.929941,0.486748,1.292662,0.291332,0.026099,0.220723,-0.255778,0.114156,-1.366022,0.366025,0.010318,1.413645
52604,-0.998809,0.422125,1.283271,0.256725,0.182065,0.191410,-0.266873,0.033949,-1.224742,0.707105,0.011332,1.413638,-0.999578,0.420170,1.285382,0.256463,0.181736,0.193429,-0.268203,0.034622,-1.224742,0.707105,0.011332,1.413638,-0.999458,0.419733,1.284388,0.255869,0.182562,0.193297,-0.267041,0.035450,-1.224742,0.707105,0.011332,1.413638,-0.999127,0.419665,1.282957,0.256220,0.181318,0.192795,-0.268522,0.034331,-1.224742,0.707105,0.011332,1.413638
52605,-0.843880,0.558390,1.149796,0.246357,0.025625,0.188474,-0.347695,0.146468,-0.999998,0.999998,0.012346,1.413629,-0.844716,0.556543,1.151939,0.246147,0.025554,0.190645,-0.348561,0.147320,-0.999998,0.999998,0.012346,1.413629,-0.844453,0.556233,1.151128,0.245417,0.026491,0.190289,-0.347464,0.147937,-0.999998,0.999998,0.012346,1.413629,-0.844107,0.555681,1.149622,0.245671,0.025024,0.189965,-0.349163,0.146786,-0.999998,0.999998,0.012346,1.4136

In [56]:
#val_nse['Linear'], val_pbias['Linear'], pred, label = compa(
#    model=gru_model,df=test_df,
#    plot_col=out_features[0], target_std=target_std, target_mean=target_mean, predict_day = rnn_predict_day)

val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=test_df,
    plot_col=out_features[0], target_std=target_std, target_mean=target_mean, predict_day = 4)


print()
print()



print(val_nse['GRU'])


print(val_pbias['GRU'])
#print(pred)
#print(label)

(5261, 48)
4
(208, 168, 48)
(208, 120, 1)
(208, 120, 1)
predictions.shape = (208, 120, 1)
pred_day.shape = (208, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 4.144565
nse3 = [0.51044106]
label_day[predict_day].shape = (5, 1)
mean = 4.163009


[0.51044106]
[0.21221149]


In [58]:
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=train_df,
    plot_col=out_features[0], target_std=target_std, target_mean=target_mean, predict_day = 4)

print()
print()

print(val_nse['GRU'])
print(val_pbias['GRU'])

(42086, 48)
4
(1742, 168, 48)
(1742, 120, 1)
(1742, 120, 1)
predictions.shape = (1742, 120, 1)
pred_day.shape = (1742, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 4.2740355
nse3 = [0.70065904]
label_day[predict_day].shape = (5, 1)
mean = 4.2766314


[0.70065904]
[0.6245054]


In [59]:
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all,
    plot_col=out_features[0], target_std=target_std, target_mean=target_mean, predict_day = 4)

print()
print()

print(val_nse['GRU'])
print(val_pbias['GRU'])

(52608, 48)
4
(2180, 168, 48)
(2180, 120, 1)
(2180, 120, 1)
predictions.shape = (2180, 120, 1)
pred_day.shape = (2180, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 4.27052
nse3 = [0.6760868]
label_day[predict_day].shape = (5, 1)
mean = 4.2712


[0.6760868]
[0.57269627]
